In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from warnings import filterwarnings
filterwarnings('ignore')


In [2]:
import pandas
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
df_train_features=pd.read_csv('dengue_features_train.csv')

df_test_features=pd.read_csv('dengue_features_test.csv')

df_labels_train=pd.read_csv('dengue_labels_train.csv')

In [26]:
df_train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 18 columns):
city                                     1456 non-null object
year                                     1456 non-null int64
weekofyear                               1456 non-null int64
ndvi_ne                                  1456 non-null float64
ndvi_nw                                  1456 non-null float64
ndvi_se                                  1456 non-null float64
ndvi_sw                                  1456 non-null float64
reanalysis_air_temp_k                    1456 non-null float64
reanalysis_avg_temp_k                    1456 non-null float64
reanalysis_dew_point_temp_k              1456 non-null float64
reanalysis_max_air_temp_k                1456 non-null float64
reanalysis_min_air_temp_k                1456 non-null float64
reanalysis_precip_amt_kg_per_m2          1456 non-null float64
reanalysis_sat_precip_amt_mm             1456 non-null float64
reanalysis_speci

In [5]:
dataset_test=df_test_features

In [6]:
dataset_test.drop(columns=['week_start_date','reanalysis_relative_humidity_percent','precipitation_amt_mm','reanalysis_tdtr_k','station_min_temp_c','station_avg_temp_c'],axis=1,inplace=True)

In [7]:
dataset_test.fillna(method='ffill',inplace=True)

In [8]:
dataset_test.shape

(416, 18)

In [9]:
test=dataset_test.iloc[:,0:18].values

In [10]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder=LabelEncoder()
test[:,0]=labelencoder.fit_transform(test[:,0])

onehotencoder=OneHotEncoder(categorical_features=[1])
dataset_test=onehotencoder.fit_transform(test).toarray()

/home/nikhil97/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [11]:
test.shape

(416, 18)

In [12]:
dataset=df_train_features

In [13]:
dataset.drop(columns=['week_start_date','reanalysis_relative_humidity_percent','precipitation_amt_mm','reanalysis_tdtr_k','station_min_temp_c','station_avg_temp_c'],axis=1,inplace=True)

In [14]:
dataset.fillna(method='ffill',inplace=True)

In [15]:
dataset.shape

(1456, 18)

In [16]:
dataset.iloc[:,0].values

array(['sj', 'sj', 'sj', ..., 'iq', 'iq', 'iq'], dtype=object)

In [17]:
dataset.shape

(1456, 18)

In [18]:
X=dataset.iloc[:,0:17].values

In [19]:
X

array([['sj', 1990, 18, ..., 14.0128571429, 6.9, 29.4],
       ['sj', 1990, 19, ..., 15.3728571429, 6.37142857143, 31.7],
       ['sj', 1990, 20, ..., 16.8485714286, 6.485714285709999, 32.2],
       ...,
       ['iq', 2010, 23, ..., 17.0085714286, 6.9, 32.2],
       ['iq', 2010, 24, ..., 16.8157142857, 8.73333333333, 31.2],
       ['iq', 2010, 25, ..., 17.3557142857, 9.9, 33.7]], dtype=object)

In [20]:
y=dataset.iloc[:,17].values

In [21]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder=LabelEncoder()
X[:,0]=labelencoder.fit_transform(X[:,0])

onehotencoder=OneHotEncoder(categorical_features=[1])
dataset=onehotencoder.fit_transform(X).toarray()



/home/nikhil97/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [22]:
X

array([[1, 1990, 18, ..., 14.0128571429, 6.9, 29.4],
       [1, 1990, 19, ..., 15.3728571429, 6.37142857143, 31.7],
       [1, 1990, 20, ..., 16.8485714286, 6.485714285709999, 32.2],
       ...,
       [0, 2010, 23, ..., 17.0085714286, 6.9, 32.2],
       [0, 2010, 24, ..., 16.8157142857, 8.73333333333, 31.2],
       [0, 2010, 25, ..., 17.3557142857, 9.9, 33.7]], dtype=object)

In [23]:
y

array([16. ,  8.6, 41.4, ...,  7.4, 16. , 20.4])

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y)


In [151]:
from keras import Sequential
from keras.layers import Dense


In [165]:
def build_regressor():
    regressor = Sequential()
    regressor.add(Dense(units=15, input_dim=17,activation='relu'))   ##20,
    regressor.add(Dense(units=12,activation='relu'))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units=12,activation='relu'))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units=10,activation='relu'))
    regressor.add(Dense(units=8,activation='relu'))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units=6,activation='relu'))
    regressor.add(Dropout(0.1))
    regressor.add(Dense(units=6,activation='relu'))
    regressor.add(Dense(units=1))
    regressor.compile(optimizer='adam', loss='mean_squared_error',  metrics=['mae'])
    return regressor
    

In [166]:
from keras.wrappers.scikit_learn import KerasRegressor
regressor = KerasRegressor(build_fn=build_regressor,batch_size=5,epochs=100)

In [167]:
results=regressor.fit(X_train,y_train)

Epoch 1/100
1092/1092 [==============================] - 1s 1ms/step - loss: 8107.1044 - mean_absolute_error: 61.8450
Epoch 2/100
1092/1092 [==============================] - 0s 303us/step - loss: 3046.3192 - mean_absolute_error: 36.5288
Epoch 3/100
1092/1092 [==============================] - 0s 285us/step - loss: 2843.0960 - mean_absolute_error: 35.2246
Epoch 4/100
1092/1092 [==============================] - 0s 304us/step - loss: 2742.9122 - mean_absolute_error: 35.0106
Epoch 5/100
1092/1092 [==============================] - 0s 288us/step - loss: 2733.4103 - mean_absolute_error: 34.1090
Epoch 6/100
1092/1092 [==============================] - 0s 349us/step - loss: 2659.8302 - mean_absolute_error: 33.9368
Epoch 7/100
1092/1092 [==============================] - 0s 296us/step - loss: 2580.9863 - mean_absolute_error: 32.3723
Epoch 8/100
1092/1092 [==============================] - 0s 359us/step - loss: 2583.7312 - mean_absolute_error: 33.0513
Epoch 9/100
1092/1092 [===================

In [27]:
y_pred= regressor.predict(X_test)

In [28]:
res=regressor.predict(test).astype(int)

In [29]:
submission = pd.read_csv("submission_format.csv",
                         index_col=[0, 1, 2])

submission.total_cases = res
submission.to_csv("new_Values_neural.csv")